# What Western companies are leaving Russia?

*March 18, 2022*

Came across [some interesting data](https://som.yale.edu/story/2022/over-400-companies-have-withdrawn-russia-some-remain) today showing which companies are leaving Russia, and in what way they're leaving (are they fully withdrawing? Permanently? Will they be back?). It's light on actual numbers, but I thought I'd take a look at how you can visualize very text-heavy data in Datawrapper. [Here](https://www.datawrapper.de/_/lrj4C/)'s the final product so you can see what it is I'm working towards!

Start with Pandas and by reading in the data.

In [79]:
import pandas as pd

raw = pd.read_csv("../raw/RAW 2022 COMPANIES LEAVING RUSSIA.csv", skiprows=3).iloc[:,:5].dropna(how="all")
raw["category"] = pd.Series()

raw.columns = raw.columns.str.strip()

display(raw)

C:\Users\dexmc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\dexmc\AppData\Local\Temp/ipykernel_16080/1264870985.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  raw["category"] = pd.Series()


,Name,Logo,Action,Date of Last Action,Link to Announcement,category
0,Aalberts,NaN,still operating in Russia,NaN,NaN,NaN
1,Acer,NaN,still operating in Russia,NaN,NaN,NaN
2,Alibaba,NaN,still operating in Russia,NaN,NaN,NaN
3,Align Technology,NaN,still operating in Russia,NaN,NaN,NaN
4,Asus,NaN,citing conditions for effective standstill in ...,NaN,NaN,NaN
...,...,...,...,...,...,...
724,World Rowing Federation,NaN,ban on all Russian competition,March 1,https://worldrowing.com/2022/03/01/world-rowin...,NaN
725,World Rugby Union,NaN,ban on all Russian competition,March 1,https://www.dailymail.co.uk/sport/sportsnews/a...,NaN
726,WPP PLC,NaN,suspend all operations in Russia,March 4,https://www.reuters.com/world/uk/british-ad-gr...,NaN
727,WWE,NaN,suspend all operations in Russia,March 3,https://www.youtube.com/watch?v=5h5AQn4ryXQ,NaN


This data needs a LOT of cleaning, so let's get started. First we copy the raw so we can work with it a bit.

In [80]:
data = raw.copy()


We need to expand the header rows into a new column called "category", then remove those header rows. First, we isolate those rows.

In [81]:
header_rows = data.loc[data["Name"].str.contains("BUYING|SCALING|SUSPENSION|WITHDRAW", regex=True), :]

header_rows

,Name,Logo,Action,Date of Last Action,Link to Announcement,category
95,BUYING TIME – Holding Off New Investments/Deve...,NaN,NaN,NaN,NaN,NaN
197,SCALING BACK – Reducing Current Operations (35...,NaN,NaN,NaN,NaN,NaN
300,SUSPENSION – Keeping Options Open for Return (...,NaN,NaN,NaN,NaN,NaN
553,"WITHDRAWAL - Clean Break - Surgical Removal, R...",NaN,NaN,NaN,NaN,NaN


Then, we grab just the category - the first bit of these header rows - and store it in it's own column.

In [82]:
header_rows["category"] = header_rows["Name"].str.replace("–", "-").str.split("-").str[0]



C:\Users\dexmc\AppData\Local\Temp/ipykernel_16080/3439377241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  header_rows["category"] = header_rows["Name"].str.replace("–", "-").str.split("-").str[0]


Then we take that slice and replace the rows in our dataset with them.

In [83]:
data.loc[data["Name"].str.contains("BUYING|SCALING|SUSPENSION|WITHDRAW", regex=True), :] = header_rows
data.loc[data["Name"].str.contains("BUYING|SCALING|SUSPENSION|WITHDRAW", regex=True), ["Name", "Action", "category"]]

,Name,Action,category
95,BUYING TIME – Holding Off New Investments/Deve...,NaN,BUYING TIME
197,SCALING BACK – Reducing Current Operations (35...,NaN,SCALING BACK
300,SUSPENSION – Keeping Options Open for Return (...,NaN,SUSPENSION
553,"WITHDRAWAL - Clean Break - Surgical Removal, R...",NaN,WITHDRAWAL


In [84]:
data["category"] = data["category"].ffill().fillna("DIGGING IN").str.capitalize()
data["Action"] = data["Action"].str.capitalize().str.replace("russia", "Russia").str.replace("belarus", "Belarus").str.strip() + "."
data = data[~data["Name"].str.contains("BUYING|SCALING|SUSPENSION|WITHDRAW|Name")].sort_values("Name")

data

,Name,Logo,Action,Date of Last Action,Link to Announcement,category
302,3M,NaN,Suspend operations in Russia.,March 10,https://apnews.com/article/russia-ukraine-mosc...,Suspension
315,AB InBev,NaN,Suspend licenses for production and sales in R...,March 11,https://www.ab-inbev.com/news-media/news-stori...,Suspension
99,ADM,NaN,Unspecified scaling down of non-essential oper...,March 11,https://www.adm.com/news/news-releases/adm-exp...,Buying time
306,ADP,NaN,Suspend new sales/services to Russia.,March 9,https://www.adp.com/about-adp/statement-on-ukr...,Suspension
559,AECOM,NaN,Suspend operations in Russia.,March 7,https://aecom.com/press-releases/aecom-to-exit...,Withdrawal
...,...,...,...,...,...,...
42,Young Living,NaN,Still operating in Russia.,NaN,NaN,Digging in
233,Yum Brands,NaN,Suspend operations of company-owned restaurant...,March 8,https://www.yum.com/wps/portal/yumbrands/Yumbr...,Scaling back
495,ZHA,NaN,Suspend operations in Russia.,March 5,https://www.architectsjournal.co.uk/news/chipp...,Suspension
13,dōTERRA,NaN,Still operating in Russia.,NaN,NaN,Digging in


In [85]:

data.groupby("category").count()

,Name,Logo,Action,Date of Last Action,Link to Announcement
category,,,,,
Buying time,58,0,58,58,58
Digging in,43,0,43,0,0
Scaling back,35,0,35,35,35
Suspension,194,0,194,192,192
Withdrawal,174,0,174,174,174


We'll iterate through all the different types of action on Russia (everything in the "Type" column). First, we make a new empty dataframe where the new text strings will go.

In [86]:
table = pd.DataFrame({"Companies": []})

 We can get all unique values in a column very easily as a list.

In [87]:
types = data["category"].unique()

types

array(['Suspension ', 'Buying time ', 'Withdrawal ', 'Digging in',
       'Scaling back '], dtype=object)

Finally, we iterate through the types, finding all companies that match a type and joining them together into a long string, in this case each one separated by a comma and a space.

In [89]:
for type in types:
    table.loc[type, "Companies"] = ", ".join(data.loc[data["category"] == type, "Name"].str.strip())
    
display(table)

,Companies
Suspension,"3M, AB InBev, ADP, Abrdn, Adidas, Adobe, Airbu..."
Buying time,"ADM, Abbott Labs, Accor, Air Liquide, Amerisou..."
Withdrawal,"AECOM, AMD, Accenture, Acronis, Activision Bli..."
Digging in,"Aalberts, Acer, Alibaba, Align Technology, Asu..."
Scaling back,"Abbvie, Allianz, Amadeus IT Group, BNP Paribas..."


You can see above how the companies have been joined together. Now you can [go over to the visualization](https://www.datawrapper.de/_/lrj4C/) to see how I used this output to visualize these companies.

### Take II: A searchable table in Datawrapper

In [93]:
searchable = data.copy()

searchable[""] = (data["category"]
                 .str.replace("Suspension", "❌")
                 .str.replace("Buying time", "❌")
                 .str.replace("Withdrawal", "❌")
                 .str.replace("Digging in", "🟢")
                 .str.replace("Scaling back", "❌")
                 )

searchable = searchable[["Name", "category", "Action", ""]].set_index("Name")

searchable

,category,Action,
Name,,,
3M,Suspension,Suspend operations in Russia.,❌
AB InBev,Suspension,Suspend licenses for production and sales in R...,❌
ADM,Buying time,Unspecified scaling down of non-essential oper...,❌
ADP,Suspension,Suspend new sales/services to Russia.,❌
AECOM,Withdrawal,Suspend operations in Russia.,❌
...,...,...,...
Young Living,Digging in,Still operating in Russia.,🟢
Yum Brands,Scaling back,Suspend operations of company-owned restaurant...,❌
ZHA,Suspension,Suspend operations in Russia.,❌


[Here](https://www.datawrapper.de/_/7Gc6h/)'s the table in action in Datawrapper.

\-30\-